# Least Dangerous Path
Introdução a Otimização (COM361)

Professor Amit Bhaya 
2022.2

Henrique Marques Lozano (henriquemarques@poli.ufrj.br), Matheus Gomes Rocha (matheusxvtb@poli.ufrj.br)


### Índice

1. Introdução
2. Modelo Matemático
3. Solução
4. Resultados e Discussão
5. Subseção Opcional
6. Conclusão
7. Referências bibliográficas

## 1. Introdução ##

É notório que, há noticias sobre acidentes envolvendo rotas perigosas na televisão, tal qual, um Uber entrando em zonas perigosas no estado do Rio de Janeiro: 

-[homem e morto a tiros em acesso ao complexo do chapadao](https://g1.globo.com/rj/rio-de-janeiro/noticia/2022/10/27/homem-e-morto-a-tiros-em-acesso-ao-complexo-do-chapadao.ghtml). 

Famílias entrando em zonas gélidas e perdendo o controle do carro e consequentemente sofrendo um acidente: 

-[Woman killed after her car loses control on icy I-35W ramp in Minneapolis](https://www.cbsnews.com/minnesota/news/woman-killed-after-her-car-loses-control-on-i-35w-ramp-in-minneapolis/). 

Ou caminhos onde o trajeto é totalmente inacessível pois há muitos locais danificados pelo tempo, a ideia primordial por trás desse trabalho é criar um algoritmo que consiga traçar rotas seguras e eficientes independentes do relevo, seja ele alto, baixo, obtuso, entre outras formas de relevo e com menor custo e risco para o usuário, de forma que ela consiga sair do seu ponto inicial e ir até o seu ponto final de destino da melhor forma possível, e claro, segura.


Vamos iniciar a criação do código da seguinte forma, primeiros criaremos o terreno no qual será feito os testes, será feita uma analise minuciosa para decidir quais serão as restrições do problema, por exemplo: Terrenos muito íngremes, com formatos muito repetidos, cumes muito acentuados e crateras, ou então, rever se o algoritmo está criando um caminho possível de ser realizado, cada situação será analisada de forma prática para que se assemelhe a realidade. Será de suma importância a utilização do Algoritmo de Dijkstra para cálculo do Caminho de Custo Mínimo. Com o algoritmo de Dijkstra, você poderá encontrar o caminho de menor custo entre nós em um grafo. Particularmente, você poderá encontrar o caminho de menor custo entre um nó (denominado “origem”) e todos os outros nós do grafo, produzindo uma árvore de custo mínimo. Este algoritmo é usado em dispositivos GPS para encontrar o caminho mais curto entre a localização atual e o destino.




## 2. Modelo matemático ##

O problema do caminho mais curto/seguro é talvez o problema mais famoso e importante na otimização de redes. Ele encontra um caminho de um determinado nó de origem para um determinado nó de destino com o/a menor/maior custo/segurança de/do caminho. Mas, no nosso caso, vamos utilizar a altura do nosso relevo, considere como se fosse o esforço para conseguir subir um degrau, ou como no exemplo abaixo, o senhor Gato quer pegar o senhor Rato, mas, ele apenas pode pular em uma altura de 1 bloco, pois, se caso tente pular em um bloco que possui 2 de altura, ele certamente cairá e vai se machucar (Por motivos médicos, o Senhor Gato não consegue cair de pé), logo, o caminho mais seguro a se fazer é 'A -> D -> E -> C', esse é um caso especial do problema de fluxo de rede de custo mínimo; portanto, o problema do caminho mais curto pode ser resolvido como um problema de programação linear, e utilizando o Algorítmo de Dijkstra. A partir da matriz discreta do terreno, devemos entender, qual o custo de deslocar-se de um ponto A a outro B. Assumindo MRU sem atrito, a unica variacao de energia ocorre com a mudanca de altura. Para esse caso, subir uma altura $ h $ custa a mesma energia que descer ela. Com isso em mente, geramos uma matriz quadrada com linhas que representam os nos de origem e as colunas os nos de saida. Essa matriz é simetrica na diagonal e preenchida pelas diferencas de altura entre pontos adjacentes. um exemplo desse problema é:


![fixit flowchart][Esquema, exemplo]

[Esquema, exemplo]: https://user-images.githubusercontent.com/81640187/209411044-c5027f47-23cf-454d-aa22-4fb168731343.png

Abaixo, há um modelo matemático simples de problema de custo/esforço mínimo que utilizamos como base:

$$
\begin{aligned}
\min_{x} \sum_{i=1}^n \sum_{j \in S(i)} c_{ij} x_{ij} \\
\end{aligned}
$$

Sujeito a:
$$
\begin{aligned}
\sum_{j \in S(1)} x_{1j} = 1 
\end{aligned}
\\

\begin{aligned}
\sum_{i \in A(n)} x_{in} = 1
\end{aligned}
\\

\begin{aligned}
\sum_{i \in A(j)} x_{ij} = \sum_{k \in S(j)} x_{jk}, j = 2, ..., n-1. 
\end{aligned}

$$

$$
\begin{aligned}
     \text{Esse modelo matemático para o problema de caminho menos custoso do nó 1 de origem até o nó $ n $, de um grafo qualquer G=(N, E), N={1, 2, ..., n.} }
\end{aligned}
$$


A variável escolhida é :
\begin{aligned} x_{ij} \in (0, 1) \implies \text{Ativação, ou não do arco(i, j)}
\end{aligned}

Com os parâmetros :
\begin{aligned} c_{ij} \implies \text{Custo unitário do fluxo em(i, j)}
\end{aligned}

\begin{aligned} S_{j} \implies \text{É o conjuto dos nós subsequentes de j}
\end{aligned}

\begin{aligned} A_{j} \implies \text{É o conjuto dos nós anteriores de j}
\end{aligned}

Com toda a modelagem feita, colocaremos o algoritmo de Dijkstra em ação, tal esse que é a solução para o problema em que foi proposto, de caminho de mínimo esforço. Funciona fundamentalmente em grafos não orientados e orientados, mas, as arestas em questão apenas podem possuir valores positivos. 

Para a entrada, deverá ser um Grafo ponderado:

\begin{aligned}
 G=(N, E) 
\end{aligned}
E nó origem: 
\begin{aligned}
 O \in N 
\end{aligned}
De modo que todos os custos entre as arestas sejam positivos.

A saída será o comprimento do caminho menos custoso de um nó selecionado de origem:
\begin{aligned}
 O \in N 
\end{aligned}
Para todos os outros nós em questão.

E esse fantástico algoritmo funciona da seguinte forma, ele identifica, a partir do nó de origem qual vai ser o caminho menos custoso entre ele e todos os outros do grafo. No começo, o conjunto de nós subsequentes possuem apenas o nó inicial de origem, na medida que ocorre um avanço no grafo, ele seleciona o próximo nó que está mais perto do inicial, após selecionar, a distancia entre cada nó é atualizada, com base na distancia em relação à origem, com base nesse novo ponto, a distância relativa entre nó final vai ficando cada vez menor, e essa é a sua nova posição.
É notório que, após escolher um nó como origem da pesquisa, o algoritmo de Dijkstra calcula o caminho de menor esforço entre esse nó até os subsequentes do grafo. O procedimento é iterativo entre cada etapa e determinando, na interação 1, o nó mais próximo e menos custoso do nó de origem O, nas proximas iterações, o segundo nó mais próximo do nó O, e assim consequitivamente, até que finalmente se alcance o ponto de de destino, após n iterações.
Considerando que há um grafo orientado G=(N, E) e p é um nó desse grafo G, inicialmente devemos colocar um valor zero à cada estimativa do esforço mínimo do nó de origem e infinito às demais estimativas.

Demonstração abaixo:

Onde a imagem (1) é a situação inicial, (2) intermediaria e (3) final.
![FINAL](https://user-images.githubusercontent.com/81640187/209583441-fddfd121-24d9-483c-8bcc-18a61f875a9a.png)

## 3. Solução ##

Nesta seção, coloque seu código em Julia + JuMP e resolva o problema proposto. Seu código deve ser limpo (não macarrônico!), de fácil leitura, bem comentado e anotado e deve compilar sem erros em Julia 1.x, x$\geq 1$! Não valem códigos em outras linguagens. **Vou rodar seu código para avaliar seu projeto**. Sugiro a utilização de múltiplos blocos de códigos separados por blocos de texto (células Markdown) explicando as várias partes da sua solução. Sugiro também a resolução de várias versões do seu problema, com modelos e hipóteses diferentes.

É permitido chamar pacotes externos, mas evite a utilização de bibliotecas exóticas (pois, em geral, não rodam em todas as versões de Julia, e terei que instalar a mesma versão que você usou, ou rodar na plataforma Google Colab, que gostaria de evitar).

In [3]:
# Este é um exemplo de um bloco de código
using JuMP, Clp
m = Model(with_optimizer(Clp.Optimizer,LogLevel=0) )
bichos = [:cavalos, :jegues, :cabras]  # estes são os bichos 
@variable(m, x[bichos] >= 0)          # as quantidades de cada um (não podem ser negativas)
@constraint(m, sum(x) <= 10)          # não podemos ter mais de 10 no total.
@objective(m, Max, x[:cavalos])        # queremos maximizar o número de cavalos
optimize!(m)

for i in bichos
    println("O número total de ", i, " é: ", JuMP.value.(x[i]))     # imprime o resultado na tela
end

O número total de cavalos é: 10.0
O número total de jegues é: 0.0
O número total de cabras é: 0.0


**Tenha certeza de que seu código compila corretamente! Rodarei seu código!**

## 4. Resultados e discussão ##

Neste seção, os resultados obtidos serão exibidos e discutidos. Mostre figuras, gráficos, imagens, curvas de compromisso, e o que mais puder melhor ilustrar seus resultados. A discussão deverá explicar o que significam os resultados e como interpretá-los. As limitações da sua abordagem/modelo também devem ser colocadas, bem como uma análise da sensibilidade dos resultados em relação às hipóteses feitas.


Utilize plots (veja exemplos  `PyPlot` [aqui](https://gist.github.com/gizmaa/7214002))

Aqui está um exemplo de uma tabela (em Markdown):

| Tabelas        | São           | Boas  |
| ------------- |:-------------:| -----:|
| col 3 é      | alinhado à direita |\$1600 |
| col 2 é      | centrado      |  \$12 |
| texto | também serve      |   \$1 |

### 4.A. Subseções devem ser utilizadas para organizar seu texto.

#### 4.A.a. ou até subsubseções.

## 5. Conclusão ##

Faça um resumo do que encontrou e dos seus resultados, e fale de pelo menos uma direção na qual  seu trabalho pode ser desenvolvido no futuro, algo que poderia ser interessante em decorrência do seu projeto.


## 6. Referências bibliográficas ##

Nesta seção, cite _*todas*_ as referências utilizadas, na formulação matemática, no código ou para extrair dados ou figuras. Omissão de fontes é transgressão grave, denominada plágio.